In [12]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [13]:
orders= pd.read_excel(r'C:\\Users\\Saeid\\OneDrive\\Desktop\\DataSet\\sample-orders.xlsx')
orders.head()

,order_date,order_id,customer,grand_total
0,2009-07-11 00:00:00,CA-2011-100006,Dennis Kane,378
1,2007-08-11 00:00:00,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franzڑsisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,2004-08-11 00:00:00,CA-2011-100363,Jim Mitchum,21


In [14]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [15]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [16]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

In [17]:
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,1144,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,461,7,3050
Adam Bellavance,105,8,7756
Adam Hart,34,10,3249


In [18]:
hart = orders[orders['customer']=='Adam Hart']
hart

,order_date,order_id,customer,grand_total
703,2011-11-16,CA-2011-160066,Adam Hart,5
1606,2012-12-21,CA-2012-161795,Adam Hart,3
2371,2013-06-27,CA-2013-144015,Adam Hart,262
2429,2013-12-18,CA-2013-147109,Adam Hart,217
2473,2013-09-16,CA-2013-149797,Adam Hart,842
3045,2014-09-26,CA-2014-112004,Adam Hart,199
3175,2014-04-16,CA-2014-118857,Adam Hart,196
3315,2014-10-24,CA-2014-125451,Adam Hart,1170
3719,2014-05-20,CA-2014-145702,Adam Hart,342
4101,2014-11-27,CA-2014-165029,Adam Hart,13


In [19]:
(NOW - dt.datetime(2014,11,27)).days==34

True

In [20]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

quantiles

,recency,frequency,monetary_value
0.25,44.0,5.0,1145.0
0.50,154.0,6.0,2257.0
0.75,404.0,8.0,3784.0


In [21]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.25: 44.0, 0.5: 154.0, 0.75: 404.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

In [22]:
rfmSegmentation = rfmTable

In [23]:
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [24]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [25]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [26]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,1144,3,887,4,4,4,444
Aaron Hawkins,12,7,1744,1,2,3,123
Aaron Smayling,461,7,3050,4,2,2,422
Adam Bellavance,105,8,7756,2,2,1,221
Adam Hart,34,10,3249,1,1,2,112
